In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import collections
import seaborn as sns
from tqdm.notebook import tqdm
import requests
import lxml
import pickle
from bs4 import BeautifulSoup

In [14]:
import _pickle as cPickle

In [6]:
f = open('/home/harsh.d/Shoaib/Benchmark Data/PwC Metadata/PwC_Dataset_2')
data = json.load(f)

In [4]:
with open('/home/harsh.d/Shoaib/Benchmark Data/PwC Metadata/papersWithAbstract') as f:
    ppr = json.load(f)

In [9]:
df = pd.DataFrame(ppr)

In [10]:
df = df.filter(['arxiv_id','title','date','abstract'])
df = df.dropna()
df['title'] = df['title'].apply(str.lower)
df['abstract'] = df['abstract'].apply(str.lower)
df

,arxiv_id,title,date,abstract
0,1805.10616,dynamic network model from partial observations,2018-05-27,can evolving networks be inferred and modeled ...
1,1806.06827,pac-bayes bounds for stable algorithms with in...,2018-06-18,pac-bayes bounds have been proposed to get ris...
2,1806.06820,automated bridge component recognition using v...,2018-06-18,this paper investigates the automated recognit...
3,1802.06093,gradient descent with identity initialization ...,2018-02-16,we analyze algorithms for approximating a func...
4,1806.06811,temporal coherence-based self-supervised learn...,2018-06-18,in order to provide the right type of assistan...
...,...,...,...,...
343301,2301.11262,better than dfa? a bayesian method for estimat...,2023-01-26,detrended fluctuation analysis (dfa) is the mo...
343302,2301.10865,persistent topological laplacian analysis of s...,2023-01-25,topological data analysis (tda) is an emerging...
343303,2301.10869,a deep neural network algorithm for linear-qua...,2023-01-25,we analyze a fixed-point algorithm for reinfor...
343304,2301.11078,new developments in econophysics: option prici...,2023-01-25,we synthesize and discuss some new development...


In [ ]:
url = 'https://paperswithcode.com/sota/image-classification-on-mnist'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
f = requests.get(url, headers = headers)
soup = BeautifulSoup(f.content, 'html.parser')
tab = soup.find('script', id="evaluation-table-data")
x = tab.contents[0]
x = x.replace('false',"False")
x = x.replace('true',"True")
x = x.replace('null',"None")
x = eval(x)
titles = []
urls = []
arxiv_ids = []
metrics = []

for x_s in x:
    if x_s['paper']['title'] is not None:
        titles.append(x_s['paper']['title'].lower())
        urls.append("https://paperswithcode.com"+x_s['paper']['url'])
        metrics.append(x_s['metrics'])
index = 0
for t in titles:
    if len(df[df["title"] == t]) >0:
        arxiv_ids.append(df[df["title"] == t]['arxiv_id'].values[0])
    else:
        arxiv_ids.append(id_locator(urls[index]))
    index+=1
        
fft = {'arxiv id':arxiv_ids,'Paper Titles':titles,'Metrics': metrics}

In [2]:
def id_locator(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    f = requests.get(url, headers = headers)
    movies_lst = []
    soup = BeautifulSoup(f.content, 'html.parser')
    if soup.find('a',{"class":"badge badge-light"}) is None:
        return 0
    if "arxiv"  in soup.find('a',{"class":"badge badge-light"})['href']:
        return soup.find('a',{"class":"badge badge-light"})['href'][22:-6]
    return 0


In [3]:
def generate_tables_for_benchmars(lin):
    url = lin
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    f = requests.get(url, headers = headers)
    soup = BeautifulSoup(f.content, 'html.parser')
    tab = soup.find('script', id="evaluation-table-data")
    if tab is None:
        return
    x = tab.contents[0]
    x = x.replace('false',"False")
    x = x.replace('true',"True")
    x = x.replace('null',"None")
    x = x.replace('NaN',"None")
    x = eval(x)
    titles = []
    urls = []
    arxiv_ids = []
    metrics = []

    for x_s in x:
        if x_s['paper']['title'] is not None:
            titles.append(x_s['paper']['title'].lower())
            urls.append("https://paperswithcode.com"+x_s['paper']['url'])
            metrics.append(x_s['metrics'])
    index = 0
    for t in titles:
        if len(df[df["title"] == t]) >0:
            arxiv_ids.append(df[df["title"] == t]['arxiv_id'].values[0])
        else:
            arxiv_ids.append(id_locator(urls[index]))
        index+=1

    fft = {'arxiv id':arxiv_ids,'Paper Titles':titles,'Metrics': metrics}
    return fft

In [4]:
def task_generator(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'}
    f = requests.get(url, headers = headers)
    soup = BeautifulSoup(f.content, 'html.parser')
    try:
        tbl = [tag.find("a")["href"] for tag in soup.select("td:has(a)")]
        sota_tbl = set()
        for x in tbl:
            if "sota" in x:
                sota_tbl.add("https://paperswithcode.com"+x)
    except Exception as e:
        print(e)
        count+=1
    return sota_tbl

In [9]:
# data

In [7]:
dataset_urls = []
for x in tqdm(data):
    dataset_urls.append(x['url'])

  0%|          | 0/7745 [00:00<?, ?it/s]

In [11]:
len(dataset_urls)

7745

In [12]:
import multiprocessing
outputs=[]
def driver_func():
    # Note: Keep Num_Process <= 45 to avoid system hang
    PROCESSES = 2
    with multiprocessing.Pool(PROCESSES) as pool:
        results=[pool.apply_async(task_generator, args=(p,)) for p in tqdm(dataset_urls[:5])]
        for r in tqdm(results):
            try:
                if(r.get()!=None):
                    outputs.append(r.get())
                
            except Exception as e:
                    pass
        pool.close()
        pool.join()
    
driver_func()

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
dataset_and_task = {}
for i in tqdm(range(len(dataset_urls))):
    dataset_and_task[dataset_urls[i]] = outputs[i]

In [19]:
dataset_and_task

{'https://paperswithcode.com/dataset/mnist': {'https://paperswithcode.com/sota/anomaly-detection-on-mnist',
  'https://paperswithcode.com/sota/anomaly-detection-on-mnist-test',
  'https://paperswithcode.com/sota/classification-with-binary-weight-network-on-3',
  'https://paperswithcode.com/sota/clustering-algorithms-evaluation-on-mnist',
  'https://paperswithcode.com/sota/continual-learning-on-rotated-mnist',
  'https://paperswithcode.com/sota/continuously-indexed-domain-adaptation-on-3',
  'https://paperswithcode.com/sota/core-set-discovery-on-mnist',
  'https://paperswithcode.com/sota/deep-clustering-on-mnist',
  'https://paperswithcode.com/sota/density-estimation-on-mnist',
  'https://paperswithcode.com/sota/density-estimation-on-mnist-conditional',
  'https://paperswithcode.com/sota/domain-adaptation-on-mnist-to-usps',
  'https://paperswithcode.com/sota/domain-adaptation-on-rotating-mnist',
  'https://paperswithcode.com/sota/domain-adaptation-on-svnh-to-mnist',
  'https://paperswit

In [45]:
leaderboard = {}
for dataset in tqdm(dataset_and_task):
    ttsskk = {}
    for task in dataset_and_task[dataset]:
        pop = generate_tables_for_benchmars(task)
        ttsskk[task] = pop
    leaderboard[dataset] = ttsskk

  0%|          | 0/7745 [00:00<?, ?it/s]

In [46]:
len(leaderboard)

7744

In [50]:
with open('leaderboard_details', 'wb') as config_dictionary_file:
  pickle.dump(leaderboard, config_dictionary_file)